In [2]:
import mysql.connector

cnx = mysql.connector.connect(user='mm_team03_01', password='mm_team03_01Pass-',
                              host='CSSQL',
                              database='mm_team03_01')

To reader: please restart the kernal and click "run all" in order to run the program.

In [3]:
def app_selector():
    print("Would you like to scrape data or perform a query? Enter 'scrape' or 'query'")
    q = input().strip().lower()  # Convert to lowercase for case-insensitive comparison
    print("Input received")

    if q == "scrape":
        print("Please enter the URL you want to scrape from as a string, and I will scrape the address from that website and enter it into the database")
        s = input()
        address_scrape(s)
    elif q.strip('"') == "query":
        print("Please select the query you would like to perform by entering the appropriate integer:")
        print("1: Search for all films by a certain film director")
        print("2: Search for the highest rated film of the month given")
        print("3: See how many tickets a particular theater has sold")
        print("4: Filter movies based on their runtime to watch a movie with your preferred length")
        print("5: Curious about theaters? Use theater ID to look up more info")
        print("6: Curious about movies? Use movie ID to look up more info")
        print("7: Enter admin mode. For admin use only")
        s = input()
        try:
            query_selector(int(s))
        except ValueError:
            print("Invalid input. Please enter an integer.")
    else:
        print("Invalid input")

# Define your address_scrape and query_selector functions here


In [4]:
def data_app():
    while True:
        app_selector()  # Assuming app_selector() is defined elsewhere
        print("Are you finished? Enter Y to end or enter N to continue")
        w = input().strip().upper()  # Convert to uppercase for case-insensitive comparison
        if w == "Y":
            print("Thank you")
            return None  # Using None instead of none
        elif w != "N":
            print("Invalid input. Please select Y or N.")
        # No need for another loop for N case, as it will go back to the beginning of the loop.

# Example of how to use the data_app function
#data_app()


In [5]:
def query_selector(q):
    if q==3:
        print("See how busy a theater is by checking how many tickets it has sold")
        busy_theaters()
    if q==7:
        print("Entering admin mode. Please enter your password")
        w=input()
        if w.strip()=="password":
            print("Access granted")
            print("Select queries to perform")
            admin_mode()
        else:
            print("Access denied")
    elif q==5:
        print("Enter the theater ID to find out about its features")
        get_theater_info()
    elif q==6:
        print("Enter the movie ID to find out more  about it")
        get_movie_info()
    elif q==4:
        print("Have a time crunch? See get some options for shorter movies")
        time_limit_movies()
    elif q==2:
        print("Check out the most popular movies of each month by entering the month number")
        month_top_movie()
    elif q==1:
        print("Find movies by your favorite director")
        director_query()

In [13]:
def admin_mode():
    import csv
    import mysql.connector
    print("Admin mode initialized. Select your query by entering a number:")
    print("1: Recent Customer Lookup. Find the most recent 10 ticket holders for your theater to email promotions to")
    print("2: Customer Lookup. Retrieve information for a customer given their customer ID")
    print("3: Theater Lookup. Retrieve stored information on a theater given the theater ID")
    print("4: CSV Download. Download a table from the database as a CSV file")
    r=input()
    #print(type(r))
    if int(r)==1:
        
        connection = mysql.connector.connect(user='mm_team03_01', password='mm_team03_01Pass-',
                              host='CSSQL',
                              database='mm_team03_01')
        cursor = connection.cursor()
        print("Connection established! Enter the theater ID you wish to query on")
    
        p=input()
        print("You're requesting data from theater " +str(p))
        query="select customer.customer_first_name, customer.customer_last_name, customer.email from customer join online_booking on online_booking.customer_id=customer.customer_id join ticket on online_booking.ticket_id=ticket.ticket_id join theater on theater.theater_id=ticket.theater_id where theater.theater_id= %s order by online_booking.booking_date DESC limit 10;"
        parameter_values=[p.strip('""')]
# Example query: selecting data from a table
        cursor.execute(query,parameter_values)

# Fetching and printing the results
#print(cursor.fetchall())
    #(337921, 'Lassie', 'Thriller', 83, 5.42, 'Alfy Hawick')
        for row in cursor.fetchall():
    #print(row)
    #print(type(row))
    #print(row[1])
            print(row)


# Closing the cursor and connection
        cursor.close()
        connection.close()
        print("Query complete")
    elif int(r)==2:
        print("Please enter the relevant customer ID")
        get_customer_info()
    elif int(r)==3:
        print("Please enter the relevant theater ID")
        get_theater_info()
    elif int(r)==4:
        print("Downloading database to CSV. Please enter the name of the desired table")
        conn = mysql.connector.connect(user='mm_team03_01', password='mm_team03_01Pass-',
                              host='CSSQL',
                              database='mm_team03_01')


# Create cursor
        cursor = conn.cursor()
        w=input()
# Execute a query to select data from the table
        query = "SELECT * FROM {};".format(w)
        cursor.execute(query)

# Fetch all the rows
        rows = cursor.fetchall()

    #print(rows[:10])
# Close cursor and connection
        csv_file_path = 'theatertableoutput.csv'

# Write data to CSV file
        with open(csv_file_path, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
    # Write the header
            csv_writer.writerow([i[0] for i in cursor.description])
    # Write the data
            csv_writer.writerows(rows)

            print("Data exported to CSV successfully.")
        cursor.close()
        conn.close()

In [7]:
import mysql.connector

def get_movie_info():
    try:
        # Set up connection parameters
        config = {
            'user': 'mm_team03_01',
            'password': 'mm_team03_01Pass-',
            'host': 'CSSQL',  # or the IP address of your MySQL server
            'database': 'mm_team03_01',  # Name of the mm_team03_01 database
            'raise_on_warnings': True
        }

        # Connect to the database
        connection = mysql.connector.connect(**config)
        if connection.is_connected():
            # Prompt the user for movie ID
            movie_id = int(input("Enter the movie ID: "))

            # Prepare a parameterized query
            query = "SELECT movie_id, movie_title, director FROM movie WHERE movie_id = %s"

            # Execute the query with the provided movie_id as parameter
            cursor = connection.cursor()
            cursor.execute(query, (movie_id,))

            # Fetch the result
            movie_info = cursor.fetchone()

            if movie_info:
                print("movie ID:", movie_info[0])
                print("Title:", movie_info[1])
                print("director:", movie_info[2])
            else:
                print("movie not found.")

    except mysql.connector.Error as e:
        print("Error while connecting to MySQL", e)
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

# Call the function to fetch movie information
#get_movie_info() #sample input give movie ID = 8394101

In [8]:
import mysql.connector

def get_theater_info():
    try:
        # Set up connection parameters
        config = {
            'user': 'mm_team03_01',
            'password': 'mm_team03_01Pass-',
            'host': 'CSSQL',  # or the IP address of your MySQL server
            'database': 'mm_team03_01',  # Name of the mm_team03_01 database
            'raise_on_warnings': True
        }

        # Connect to the database
        connection = mysql.connector.connect(**config)
        if connection.is_connected():
            # Prompt the user for theater ID
            theater_id = int(input("Enter the theater ID: "))

            # Prepare a parameterized query
            query = "SELECT address, number_of_halls, hall_type_categories  FROM theater WHERE theater_id = %s"

            # Execute the query with the provided theater_id as parameter
            cursor = connection.cursor()
            cursor.execute(query, (theater_id,))

            # Fetch the result
            theater_info = cursor.fetchone()

            if theater_info:
                print("address:", theater_info[0])
                print("number_of_halls:", theater_info[1])
                print("hall_type_categories:", theater_info[2])
            else:
                print("Theater not found.")

    except mysql.connector.Error as e:
        print("Error while connecting to MySQL", e)
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

# Call the function to fetch theater information
#get_theater_info() #sample input give theater ID = 5

In [9]:
import mysql.connector

def get_customer_info():
    try:
        # Set up connection parameters
        config = {
            'user': 'mm_team03_01',
            'password': 'mm_team03_01Pass-',
            'host': 'CSSQL',  # or the IP address of your MySQL server
            'database': 'mm_team03_01',  # Name of the mm_team03_01 database
            'raise_on_warnings': True
        }

        # Connect to the database
        connection = mysql.connector.connect(**config)
        if connection.is_connected():
            # Prompt the user for customer ID
            customer_id = int(input("Enter the customer ID: "))

            # Prepare a parameterized query
            query = "SELECT customer_first_name, customer_last_name, credit_card_details, email FROM customer WHERE customer_id = %s"

            # Execute the query with the provided customer_id as parameter
            cursor = connection.cursor()
            cursor.execute(query, (customer_id,))

            # Fetch the result
            customer_info = cursor.fetchone()

            if customer_info:
                print("customer_first_name:", customer_info[0])
                print("customer_last_name:", customer_info[1])
                print("credit_card_details:", customer_info[2])
                print("email:", customer_info[3])
            else:
                print("customer not found.")

    except mysql.connector.Error as e:
        print("Error while connecting to MySQL", e)
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

# Call the function to fetch customer information
#get_customer_info() #sample input give customer ID = 2 

In [10]:
def month_top_movie():
    print("Welcome to the the month's top movies query")
    import mysql.connector
    connection = mysql.connector.connect(user='mm_team03_01', password='mm_team03_01Pass-',
                              host='CSSQL',
                              database='mm_team03_01')
    cursor = connection.cursor()
    print("Connection established!")
    
    print("Input the month you desire")
    p=input()
    print("Your input was "+p)
    query="SELECT movie.movie_title FROM movie JOIN film_time_slot ON film_time_slot.movie_id = movie.movie_id WHERE MONTH(film_time_slot.date) = %s order by movie.rating DESC LIMIT 5;"
    parameter_values=[p]
# Example query: selecting data from a table
    cursor.execute(query,parameter_values)

# Fetching and printing the results
#print(cursor.fetchall())
    #(337921, 'Lassie', 'Thriller', 83, 5.42, 'Alfy Hawick')
    for row in cursor.fetchall():
    #print(row)
    #print(type(row))
    #print(row[1])
        print(row[0])


# Closing the cursor and connection
    cursor.close()
    connection.close()
    print("Query complete")

In [11]:
def director_query():
    print("Welcome to the director filter")
    import mysql.connector
    connection = mysql.connector.connect(user='mm_team03_01', password='mm_team03_01Pass-',
                              host='CSSQL',
                              database='mm_team03_01')
    cursor = connection.cursor()
    print("Connection established!")
    
    print("Please input the first and last name of the director")
    p=input()
    print("Your input was "+p)
    query="SELECT * FROM movie WHERE director =  %s;"
    parameter_values=[p.strip('""')]
# Example query: selecting data from a table
    cursor.execute(query,parameter_values)

# Fetching and printing the results
#print(cursor.fetchall())
    #(337921, 'Lassie', 'Thriller', 83, 5.42, 'Alfy Hawick')
    print("(Movie ID, Film Name, Genre, Runtime, Rating, Director)")
    for row in cursor.fetchall():
    #print(row)
    #print(type(row))
    #print(row[1])
        print(row)


# Closing the cursor and connection
    cursor.close()
    connection.close()

In [12]:
##I'm gonna make the scrape function

def address_scrape(url):
    import requests
    from bs4 import BeautifulSoup
    response = requests.get(url)
    html_content = response.content

# Load HTML content into Beautiful Soup
    soup = BeautifulSoup(html_content, 'html.parser')

# Find the h2 tag containing "How to find us"
    address_heading = soup.find('h2', text='How to find us')
# Check if the address_heading is found
    if address_heading:
    # Extract the divs containing the address details
        address_divs = address_heading.find_next_siblings('div')

    # Extract street address, city, and zip code from the divs
        street_address = address_divs[0].get_text()
        city = address_divs[1].get_text()
        address_success=street_address+", "+ city
        print(address_success)
        connection = mysql.connector.connect(user='mm_team03_01', password='mm_team03_01Pass-',
                              host='CSSQL',
                              database='mm_team03_01')
        cursor = connection.cursor()
        cursor.execute("SELECT MAX(theater_id) FROM theater")
        max_theater_id = cursor.fetchone()[0]

# Increment the max theater_id by 1
        new_theater_id = max_theater_id + 1 if max_theater_id is not None else 1

# Example query: selecting data from a table
        new_entry_values = {
        "theater_id":new_theater_id,
        "address": address_success,
        "number_of_halls": 1,
        "hall_type_categories": "Standard"
        }
        sql_query = "INSERT INTO theater (theater_id, address, number_of_halls, hall_type_categories) VALUES (%(theater_id)s, %(address)s, %(number_of_halls)s, %(hall_type_categories)s)"

# Execute the SQL query with the provided values
    
        cursor.execute(sql_query, new_entry_values)
        connection.commit()
        if cursor.rowcount == 1:
            print("Insert successful.")
        else:
            print("Insert failed.")
# Closing the cursor and connection
        cursor.close()
        connection.close()

    #print("Street Address:", street_address)
    #print("City:", city)
    
    else:
        print("Address not found.")

In [13]:
def busy_theaters():
    import mysql.connector
    connection = mysql.connector.connect(user='mm_team03_01', password='mm_team03_01Pass-',
                              host='CSSQL',
                              database='mm_team03_01')
    cursor = connection.cursor()
    print("Connection established!")
    print("Please enter the ID of the theater you wish to visit")
    p=input()
    print("Your input was "+p)
    query="select theater.address, count(ticket.theater_id) from theater join ticket on theater.theater_id=ticket.theater_id where theater.theater_id=%s"
    parameter_values=[p.strip('""')]
# Example query: selecting data from a table
    cursor.execute(query,parameter_values)

# Fetching and printing the results
#print(cursor.fetchall())
    #(337921, 'Lassie', 'Thriller', 83, 5.42, 'Alfy Hawick')
    for row in cursor.fetchall():
    #print(row)
    #print(type(row))
    #print(row[1])
        print("Your theater located at "+str(row[0])+" has sold "+ str(row[1])+ " tickets")


# Closing the cursor and connection
    cursor.close()
    connection.close()
    print("Query complete")

In [14]:
def time_limit_movies():
    import mysql.connector
    connection = mysql.connector.connect(user='mm_team03_01', password='mm_team03_01Pass-',
                              host='CSSQL',
                              database='mm_team03_01')
    cursor = connection.cursor()
    print("Connection established!")
    
    p=input()
    print("Your movie will be no longer than "+str(p)+ " minutes")
    query="select movie.movie_title, movie.runtime from movie where movie.runtime< %s order by movie.runtime DESC limit 10;"
    parameter_values=[p]
# Example query: selecting data from a table
    cursor.execute(query,parameter_values)

# Fetching and printing the results
#print(cursor.fetchall())
    #(337921, 'Lassie', 'Thriller', 83, 5.42, 'Alfy Hawick')
    for row in cursor.fetchall():
    #print(row)
    #print(type(row))
    #print(row[1])
        print(str(row[0])+" has a runtime of "+str(row[1]))


# Closing the cursor and connection
    cursor.close()
    connection.close()
    print("Query complete")

In [15]:
data_app()

Would you like to scrape data or perform a query? Enter 'scrape' or 'query'
Input received
Please select the query you would like to perform by entering the appropriate integer:
1: Search for all films by a certain film director
2: Search for the highest rated film of the month given
3: See how many tickets a particular theater has sold
4: Filter movies based on their runtime to watch a movie with your preferred length
5: Curious about theaters? Use theater ID to look up more info
6: Curious about movies? Use movie ID to look up more info
7: Enter admin mode. For admin use only
Entering admin mode. Please enter your password
Access granted
Select queries to perform
Admin mode initialized. Select your query by entering a number:
1: Recent Customer Lookup. Find the most recent 10 ticket holders for your theater to email promotions to
2: Customer Lookup. Retrieve information for a customer given their customer ID
3: Theater Lookup. Retrieve stored information on a theater given the theate